In [ ]:

from dataclasses import dataclass
from typing import List, Tuple

@dataclass
class Process:
    pid: str
    at: int      # Arrival Time
    bt: int      # Burst Time
    priority: int
    start: int = None
    ct: int = None   # Completion Time
    tat: int = None  # Turnaround Time
    wt: int = None   # Waiting Time
    completed: bool = False

def non_preemptive_priority_scheduling(procs: List[Process]) -> List[Tuple[str, int, int]]:
    processes = sorted(procs, key=lambda x: (x.at, x.priority, x.pid))
    n = len(processes)
    time = 0
    completed = 0
    gantt = []

    while completed < n:
        # Find all arrived processes that are not completed
        available = [p for p in processes if (p.at <= time and not p.completed)]
        if not available:
            # If no process available, jump to next arrival
            next_arrival = min(p.at for p in processes if not p.completed)
            if time < next_arrival:
                gantt.append(("idle", time, next_arrival))
            time = next_arrival
            continue


        chosen = min(available, key=lambda x: (x.priority, x.at, x.pid))


        chosen.start = time
        time += chosen.bt
        chosen.ct = time
        chosen.tat = chosen.ct - chosen.at
        chosen.wt = chosen.tat - chosen.bt
        chosen.completed = True
        completed += 1

        gantt.append((chosen.pid, chosen.start, chosen.ct))

    return gantt

def print_results(processes: List[Process], gantt: List[Tuple[str, int, int]]):
    print("\nScheduling Results (Non-preemptive Priority):\n")
    header = f"{'PID':<4}{'AT':>6}{'BT':>6}{'PR':>6}{'CT':>8}{'TAT':>8}{'WT':>8}"
    print(header)
    print("-" * len(header))

    for p in sorted(processes, key=lambda x: x.pid):
        print(f"{p.pid:<4}{p.at:>6}{p.bt:>6}{p.priority:>6}{p.ct:>8}{p.tat:>8}{p.wt:>8}")

    avg_tat = sum(p.tat for p in processes) / len(processes)
    avg_wt = sum(p.wt for p in processes) / len(processes)
    print("\nAverages:")
    print(f"Average Turnaround Time = {avg_tat:.2f}")
    print(f"Average Waiting Time    = {avg_wt:.2f}")

    print("\nGantt Chart (PID: start -> end):")
    for seg in gantt:
        print(f"  {seg[0]:<5}: {seg[1]:>3} -> {seg[2]:>3}")
    print()

if __name__ == "__main__":
    processes = [
        Process("P1", at=0,  bt=11, priority=2),
        Process("P2", at=5,  bt=28, priority=0),
        Process("P3", at=12, bt=2,  priority=3),
        Process("P4", at=2,  bt=10, priority=1),
        Process("P5", at=9,  bt=16, priority=4),
    ]

    gantt = non_preemptive_priority_scheduling(processes)
    print_results(processes, gantt)




Scheduling Results (Non-preemptive Priority):

PID     AT    BT    PR      CT     TAT      WT
----------------------------------------------
P1       0    11     2      11      11       0
P2       5    28     0      39      34       6
P3      12     2     3      51      39      37
P4       2    10     1      49      47      37
P5       9    16     4      67      58      42

Averages:
Average Turnaround Time = 37.80
Average Waiting Time    = 24.40

Gantt Chart (PID: start -> end):
  P1   :   0 ->  11
  P2   :  11 ->  39
  P4   :  39 ->  49
  P3   :  49 ->  51
  P5   :  51 ->  67

